In [2]:
import pandas as pd
import numpy as np



In [35]:
train_data=pd.read_csv("/home/anand/Kaggle/HousePrices/train.csv")
train_data_copy=train_data.copy()
out_data=pd.read_csv("/home/anand/Kaggle/HousePrices/test.csv")
#train_data=train_data.append(out_data)

In [29]:
train_data_y=train_data['SalePrice']
train_data=train_data.drop(['SalePrice'],axis=1)
train_data=train_data[out_data.columns]

cattype=[c for c in train_data.columns if train_data.dtypes[c] == 'object']
numtype=[c for c in train_data.columns if train_data.dtypes[c] != 'object']



In [30]:
drop_list=[]
for c in train_data.columns:
    if(train_data[c].isnull.sum() > 1000):        
        print(c)
        drop_list.append(c)
        print(train_data[c].isnull().sum())
drop_list

AttributeError: 'function' object has no attribute 'sum'

In [31]:
train_data['Fence'].value_counts()

MnPrv    157
GdPrv     59
GdWo      54
MnWw      11
Name: Fence, dtype: int64

In [32]:
#train_data_afterdrop=train_data.drop(drop_list,axis=1)
train_data_afterdrop=train_data.copy()

In [33]:
meanval={f:train_data_afterdrop[f].mean() for f in numtype }
train_data_afterdrop=train_data_afterdrop.fillna(value=meanval)
meanval={f:out_data[f].mean() for f in numtype }
out_data=out_data.fillna(value=meanval)

In [34]:
cattype=[c for c in train_data_afterdrop.columns if train_data.dtypes[c] == 'object']
maxvalue={f:train_data_afterdrop[f].value_counts().idxmax(axis=1) for f in cattype}
train_data_afterdrop=train_data_afterdrop.fillna(value=maxvalue)
maxvalue={f:out_data[f].value_counts().idxmax(axis=1) for f in cattype}
out_data=out_data.fillna(value=maxvalue)



In [10]:
#train_data_afterdrop[train_data_afterdrop.isnull().any(axis=1)]




In [11]:
cattype=[c for c in train_data_afterdrop.columns if train_data.dtypes[c] == 'object']
train_data_afterdrop=pd.get_dummies(train_data_afterdrop,cattype)
out_data=pd.get_dummies(out_data,cattype)

train_data_afterdrop.to_csv("Afterimpute.csv")

In [12]:

train_data_afterdrop.shape

(1460, 271)

In [13]:
final_train_x=train_data_afterdrop.drop(['Id'],axis=1)[:1400]
final_train_y=train_data_y[:1400]
final_test_x=train_data_afterdrop.drop(['Id'],axis=1)[1401:]
final_test_y=train_data_y[1401:]
from sklearn.model_selection import train_test_split
x=train_data_afterdrop.drop(['Id'],axis=1)


In [14]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn import linear_model
def plot_coeff(alphas,coefs):
    ax = plt.gca()
    ax.plot(alphas, coefs)
    ax.set_xscale('log')
    ax.set_xlim(ax.get_xlim()[::-1])  # reverse axis
    plt.xlabel('alpha')
    plt.ylabel('weights')
    plt.title('Ridge coefficients as a function of the regularization')
    plt.axis('tight')
    plt.show()

In [15]:
def plot_accuracy(alphas,accuracy):
    ax = plt.gca()
    ax.plot(alphas, accuracy)
    #ax.set_xscale('log')
    ax.set_xlim(ax.get_xlim()[::-1])  # reverse axis
    plt.xlabel('alpha')
    plt.ylabel('weights')
    plt.title('Ridge coefficients as a function of the regularization')
    plt.axis('tight')
    plt.show()

In [16]:
def Linear(type_,final_train_x,final_train_y,final_test_x,final_test_y):
    n_alphas = 200
    accuracies=[]
    coefs=[]
    alphas = np.logspace(-10, -1, n_alphas)
    if(type_=='linear'):
        reg = linear_model.LinearRegression()
        reg.fit(final_train_x,final_train_y)
        accuracy=reg.score(final_test_x,final_test_y)
        print(accuracy*100,'%')
    elif(type_=='ridge'):
        for a in alphas:
            ridge=linear_model.Ridge(alpha=a)
            ridge.fit(final_train_x,final_train_y)
            coefs.append(ridge.coef_)
            accuracies.append(ridge.score(final_test_x,final_test_y))
        plot_coeff(alphas,coefs)
        plot_accuracy(alphas,accuracies)
    elif(type_=='lasso'):
        for a in alphas:
            lasso=linear_model.Lasso(alpha=a)
            lasso.fit(final_train_x,final_train_y)
            coefs.append(lasso.coef_)
            accuracy=lasso.score(final_test_x,final_test_y)
            accuracies.append(accuracy)
        plot_coeff(alphas,coefs)
        plot_accuracy(alphas,accuracies)
    elif(type_=='lassolars'):
        for a in alphas:
            lasso=linear_model.LassoLars(alpha=a)
            lasso.fit(final_train_x,final_train_y)
            coefs.append(lasso.coef_)
            accuracy=lasso.score(final_test_x,final_test_y)
            accuracies.append(accuracy)
            print(accuracy)
        plot_coeff(alphas,coefs)
        plot_accuracy(alphas,accuracies)
    elif(type_=='br'):
        for a in alphas:
            lasso=linear_model.ElasticNet(alpha=a)
            lasso.fit(final_train_x,final_train_y)
            coefs.append(lasso.coef_)
            accuracy=lasso.score(final_test_x,final_test_y)
            accuracies.append(accuracy)
            print(accuracy)
        plot_coeff(alphas,coefs)
        plot_accuracy(alphas,accuracies)

In [17]:
final_train_x=train_data_afterdrop.drop(['Id'],axis=1)[:1400]
final_train_y=train_data_y[:1400]
final_test_x=train_data_afterdrop.drop(['Id'],axis=1)[1401:]
final_test_y=train_data_y[1401:]
x=final_train_x.copy()
y=final_train_y.copy()
x_test=final_test_x.copy()
y_test=final_test_y.copy()

In [18]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

def decomposeandrun(x,y,x_test,y_test):
    print("Standardizing...")
    scaler=StandardScaler()
    scaler.fit(x)
    x=scaler.transform(x)
    x_test=scaler.transform(x_test)
    print("Find PCA....")
    pca=PCA(0.95)
    pca.fit(x)
    x=pca.transform(x)
    x_test=pca.transform(x_test)
    gradboost(x,y,x_test,y_test)

In [19]:
from sklearn.ensemble import GradientBoostingRegressor
def gradboost(x,y,x_test,y_test):
    accuracies=[]
    lrList = range(1,20,4)
    estList = range(100,1000,100)
    maxac=0
    maxlr=0
    maxest=0
    for l in lrList:
        for est in estList:
            gb=GradientBoostingRegressor(learning_rate=l/100,n_estimators=est)
            gb.fit(x,y)
            accuracy=gb.score(x_test,y_test)
            if(maxac<accuracy):
                maxac=accuracy
                maxlr=l
                maxest=est
            accuracies.append(accuracy)
        print("Learning rate: ",maxlr/100, " Estimator: ",maxest, " Accuracu",maxac)
    print(maxac)
    print(maxlr)
    print(maxest)


In [ ]:
print(x.shape)
print(y.shape)
print(x_test.shape)
print(y_test.shape)
decomposeandrun(x,y,x_test,y_test)


In [21]:
from sklearn.feature_selection import RFE
gb=GradientBoostingRegressor(learning_rate=17/100,n_estimators=700)
selector = RFE(gb)
selector.fit(x,y)

KeyboardInterrupt: 

In [ ]:
print(x.shape)
print(y.shape)
print(x_test.shape)
print(y_test.shape)
train=selector.score(x,y)
test=selector.score(x_test,y_test)
print(train,test)

In [ ]:
gb=GradientBoostingRegressor(learning_rate=13/100,n_estimators=700)
x=train_data_afterdrop.drop(['Id'],axis=1)
y=train_data_y
x_test=out_data.drop(['Id'],axis=1)
selector = RFE(gb)
selector.fit(x,y)
result=selector.predict(x_test)

In [22]:
import xgboost
from sklearn.metrics import explained_variance_score
maxlr=0
maxg=0
maxnest=0
maxac=0
for nf in range (50, 100, 50):
    for lr in range(1,5,4):
        for g in range(1,2,1):
            nest=500
            xgb = xgboost.XGBRegressor(n_estimators=nest, learning_rate=lr/100, gamma=g, subsample=0.75,colsample_bytree=1, max_depth=50)
            selector = RFE(xgb,n_features_to_select=nf)
            selector.fit(x,y)
            yhat=selector.predict(x_test)
            ac=explained_variance_score(yhat,y_test)
            if(maxac < ac):
                maxac=ac
                maxlr=lr
                maxg=g
                maxnest=nest
            print(nf," ",lr," "," ",g," ",ac)
                

50   1     1   0.8953280307942909


In [ ]:
xgb = xgboost.XGBRegressor(n_estimators=500, learning_rate=1/100, gamma=5, subsample=0.75,colsample_bytree=1, max_depth=5)
x=train_data_afterdrop.drop(['Id'],axis=1)
y=train_data_y
x_test=out_data.drop(['Id'],axis=1)
selector = RFE(xgb,n_features_to_select=100)
selector.fit(x,y)
result=selector.predict(x_test)

In [ ]:

gboost=0
x=train_data_afterdrop.drop(['Id'],axis=1)
y=train_data_y

x_test=out_data.drop(['Id'],axis=1)
if(gboost):
    gb=GradientBoostingRegressor(learning_rate=17/100,n_estimators=700)
    scaler=StandardScaler()
    scaler.fit(x)
    x=scaler.transform(x)
    x_test=scaler.transform(x_test)
    print("Find PCA....")
    pca=PCA(0.95)
    pca.fit(x)
    x=pca.transform(x)
    x_test=pca.transform(x_test)
    gb.fit(x,y)
    
else:
    gb=GradientBoostingRegressor(learning_rate=13/100,n_estimators=700)
    gb.fit(x,y)


In [ ]:
result=gb.predict(x_test)
result

In [ ]:

out=pd.DataFrame()
out['Id']=out_data['Id']
out['SalePrice']=result
out.shape
out.to_csv("result_xgb_RFE.csv",index=False)



In [ ]:
#pca.fit(x)

#x=pca.transform(x)
#x_test=pca.transform(x_test)
Linear('linear',x,y,x_test,y_test)
#decomposeandrun(x,y,x_test,y_test)